<a href="https://colab.research.google.com/github/mjso9805/Test/blob/main/1%EC%A3%BC%EC%B0%A8_%EB%AF%B8%EB%8B%88_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_2%EB%B2%88_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 파일 업로드, 폰트 설치

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')  # 드라이브에서 파일 업로드
!sudo apt-get install -y fonts-nanum  # 폰트 다운로드. plt 사용 시 한글 깨짐 방지.
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 2. 데이터 열기

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

filename = '/content/drive/MyDrive/시군별유동인구데이터.csv' # 파일 열기
data = pd.read_csv(filename,encoding='CP949')

data.head(5)



# 3. 표 수정

In [ ]:
data['출발지(야간체류지) 시군코드'] = data['출발지(야간체류지) 시군코드'].astype(str) # 앞자리 2개 추출을 위해 문자열로 변환
data['출발 지역 코드'] = data['출발지(야간체류지) 시군코드'].str[0:2]  # 출발지 코드에서 앞 부분 2자리를 가져옴

# 모든 유입 인구 합계
data['Sum'] = (data['10대남성 유입인구'] + data['20대남성 유입인구'] + data['30대남성 유입인구'] + data['40대남성 유입인구'] + data['50대남성 유입인구'] + data['60대남성 유입인구'] + data['70대남성 유입인구'] 
               + data['10대여성 유입인구'] + data['20대여성 유입인구'] + data['30대여성 유입인구'] + data['40대여성 유입인구'] + data['50대여성 유입인구'] + data['60대여성 유입인구'] + data['70대여성 유입인구'])

# 숫자 단위를 줄이기 위해 나눔
data['Total'] = (data['Sum']/10000)

data.head()

In [ ]:
data_src = data.drop(['년월','도착지 시군코드'],axis=1)
data_dest = data.drop(['년월','출발지(야간체류지) 시군코드'],axis=1)

data_src = data_src.groupby(data_src['출발 지역 코드']).sum() # 출발지 코드 기준으로 중복 제거, 유입 인구 합계 구함
data_dest = data_dest.groupby(data_dest['도착지 시군코드']).sum() # 도착치 코드 기준으로 중복 제거, 유입 인구 합계 구함

data_src.head()

In [ ]:
data_dest.head()

# 4. 그래프에 사용할 정보 추출

In [ ]:
 # 그래프를 그리는데 필요한 출발지/ 도착지 코드와 유입 인구 합계 저장
Total_src = data_src['Total'].tolist()
local_Code = data_src.index.tolist() 

Total_dest = data_dest['Total'].tolist()
dest_Code = data_dest.index.astype(str).tolist()

print(Total_src)
print(local_Code)
print(Total_dest)
print(dest_Code)

# 5. 그래프 그리기

In [ ]:
plt.rc('font', family='NanumBarunGothic') # 폰트 적용
plt.title('출발지 기준 유동인구 그래프')
plt.bar(local_Code, Total_src)
plt.xlabel('출발지 코드')
plt.ylabel('유동 인구(단위 : 만)')
plt.show()

In [ ]:
ax = plt.subplot(1, 1, 1)
plt.rcParams["figure.figsize"] = (15,8)
plt.title('도착지 기준 유동인구 그래프')
plt.bar(dest_Code, Total_dest)
plt.xlabel('도착지 코드')
plt.ylabel('유동 인구(단위 : 만)')
plt.xticks(rotation=45)
plt.show()